# E-commerce Product Discount Validation **for Understanding Chain of Responsibility pattern**

1. **Check Product Availability(class ProductHandler):**
   - Query the database to verify if the product is still in stock.
   - If the product is out of stock, it displays an error message.

2. **Validate Discount Period(class DiscountHandler):**
   - Compare the current date with the discount start and end dates.
   - Proceed only if the current date falls within the valid discount period; otherwise, show a message.

3. **Verify Discount Value(class DiscountHandler):**
   - Retrieve the product price and discount value from the database.
   - Ensure that the discount value does not exceed the product price.
   - If it does, adjust the discount value or display a warning.

4. **Checking restrictions on certain product characteristics(class BrandHandler):**
   - Retrieve the product brand from the database.
   - Ensure that this brand is not on the prohibited list
   - If it does, display a warning.
  
5. **Notify about low stock levels for a product(class NotificationHandler)**

  
In our example, the database is replaced by a dictionary


In [49]:
import random
from datetime import datetime, timedelta
from abc import ABC, abstractmethod
from typing import Optional, Any
import logging

class Handler(ABC):
    @abstractmethod
    def set_next(self, handler: 'Handler') -> 'Handler':
        """
        Sets the next handler in the chain.
        """
        pass

    @abstractmethod
    def handle(self, request) -> Optional[str]:
        """
        Handles the request.
        """
        pass

class AbstractHandler(Handler):
    _next_handler: Handler = None

    def set_next(self, handler: Handler) -> Handler:
        """
        Sets the next handler in the chain.
        """
        self._next_handler = handler
        return handler

    def handle(self, product: Any) -> str:
        """
        Base implementation for handling requests.
        """
        if self._next_handler:
            return self._next_handler.handle(product)
        return "You can use discount for this product"

class ProductHandler(AbstractHandler):
    def handle(self, product: dict) -> str:
        """
        Handles product requests.
        """
        if product['stock'] == 0:
            return f"There's no stock for {product['name']}"
        return super().handle(product)

class BrandHandler(AbstractHandler):
    _blocked_brands = ("apple",)

    def handle(self, product: dict) -> str:
        """
        Handles brand-related requests.
        """
        if product['brand'] in self._blocked_brands:
            return f"{product['name']} is not allowed to have a discount due to brand restrictions"
        return super().handle(product)

class Discount:
    @staticmethod
    def find(discount_id: int, discounts: dict) -> dict:
        """
        Placeholder method to find a discount by its ID.
        """
        return discounts.get(discount_id, None)

class DiscountHandler(AbstractHandler):
    def handle(self, product: dict) -> str:
        """
        Handles discount-related requests.
        """
        discount = Discount.find(product['id'],discount_dict)
        if not self.has_valid_period(discount['expires_at']):
            return "The discount has expired"

        if not self.has_valid_amount(product['price'], discount['amount']):
            return "Cannot apply discount; discount value exceeds product price"
        
        return super().handle(product)

    def has_valid_period(self, discount_expires_at):
        return discount_expires_at < datetime.now()

    def has_valid_amount(self, product_price, discount_value):
        return discount_value < product_price

class NotificationHandler(AbstractHandler):
    def handle(self, product: dict) -> str:
        """
        Handles notifications for low stock.
        """
        if product['stock'] <= 10 and product['price'] >= 1000:
            logger = logging.getLogger(__name__)
            logger.info(f"{product['name']} is running out of stock; current stock: {product['stock']}")
        return super().handle(product)

# Generate random product data
def generate_random_product(product_id):
    id = product_id
    names = ["mobile phone", "laptop", "smartwatch", "tablet", "camera"]
    name = random.choice(names)  # Random product name
    stock = random.randint(0, 100)  # Random stock quantity
    brands = ["Samsung", "Apple", "Sony", "HP", "Dell"]
    brand = random.choice(brands)  # Random product brand
    price = round(random.uniform(6000, 89000), 2)  # Random product price

    return {
        'id': id,
        'name': name,
        'stock': stock,
        'brand': brand,
        'price': price,
    }

# Generate random discount data
def generate_random_discount(discount_id):
    id = discount_id
    amount = round(random.uniform(6000, 89000), 2)  # Random discount value
    expires_in_days = random.randint(-30, 30)  # Random number of days until expiration
    expires_at = datetime.now() + timedelta(days=expires_in_days)

    return {
        'id': id,
        'amount': amount,
        'expires_at': expires_at,
    }

# Create a dictionary with 5 entries
discount_dict = {}
for i in range(1,6):
    discount_id = i
    product_id = i
    discount_dict[discount_id] = generate_random_discount(discount_id)
    product_dict[product_id] = generate_random_product(product_id)

# Print the generated product data
for product_id, p_data in product_dict.items():
    print(f"Product ID {product_id}: {p_data}")
print()
    
# Print the generated discount data
for discount_id, d_data in discount_dict.items():
    print(f"Discount ID {discount_id}: {d_data}")
print()


def main():
    for product_id, product_data in product_dict.items():
        main_product_handler = ProductHandler()
        main_brand_handler = BrandHandler()
        main_discount_handler = DiscountHandler()
        main_notification_handler = NotificationHandler()

        main_product_handler.set_next(main_brand_handler).set_next(main_discount_handler).set_next(main_notification_handler)

        result = main_product_handler.handle(product_data)
        print(f"Product ID {product_id}: {result}")

if __name__ == "__main__":
    main()

Product ID 1: {'id': 1, 'name': 'laptop', 'stock': 69, 'brand': 'Samsung', 'price': 11570.95}
Product ID 2: {'id': 2, 'name': 'smartwatch', 'stock': 0, 'brand': 'Apple', 'price': 15937.38}
Product ID 3: {'id': 3, 'name': 'tablet', 'stock': 71, 'brand': 'Dell', 'price': 7893.24}
Product ID 4: {'id': 4, 'name': 'laptop', 'stock': 0, 'brand': 'Samsung', 'price': 55182.73}
Product ID 5: {'id': 5, 'name': 'camera', 'stock': 22, 'brand': 'HP', 'price': 46147.73}

Discount ID 1: {'id': 1, 'amount': 59643.38, 'expires_at': datetime.datetime(2024, 5, 4, 12, 21, 36, 781000)}
Discount ID 2: {'id': 2, 'amount': 80183.93, 'expires_at': datetime.datetime(2024, 4, 7, 12, 21, 36, 781000)}
Discount ID 3: {'id': 3, 'amount': 71087.5, 'expires_at': datetime.datetime(2024, 5, 1, 12, 21, 36, 781000)}
Discount ID 4: {'id': 4, 'amount': 88727.48, 'expires_at': datetime.datetime(2024, 5, 18, 12, 21, 36, 781000)}
Discount ID 5: {'id': 5, 'amount': 65003.84, 'expires_at': datetime.datetime(2024, 5, 18, 12, 21, 